In [41]:
"""
Скрипт для обработки данных  с портала трудвсем для БЦК
"""

'\nСкрипт для обработки данных  с портала трудвсем для БЦК\n'

In [42]:
import pandas as pd
import numpy as np
import openpyxl

In [43]:
region = 'Республика Бурятия'

In [44]:
df = pd.read_csv('data/vacancy.csv',encoding='UTF-8',sep='|',dtype=str)


In [51]:
company_df = pd.read_excel('data/company.xlsx',dtype=str)

In [52]:
company_df

,Название организации,ИНН
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""УЛАН-УДЭНСКИЙ АВИАЦИОННЫ...",0323018510
1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""УЛАН-УДЭНСКОЕ ПРИБОРОСТР...",0323053578


In [46]:
df.shape

(92697, 78)

In [47]:
df = df[df['regionName'] == region]

In [48]:
df.shape

(4116, 78)

In [49]:
df['company_inn']

78       2725026521
212      7708709686
258      0319103000
292      2460069527
304      7714731464
            ...    
92524    0326024141
92527    0323363770
92531    2460069527
92652    7810729228
92658    7810729228
Name: company_inn, Length: 4116, dtype: object

In [50]:
print('Lindy Booth!')

Lindy Booth!
